In [143]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag, ResultSet
from typing import List, Any
from datetime import datetime
import re
import load_injury_info


In [88]:
link = 'https://rotogrinders.com/lineups/nba?date=2023-01-22&site=fanduel'
a = requests.get(link)

In [190]:
def process_date(link : str) -> datetime:
    l = link.find('date=') + 5
    r = link.rfind('&')
    txt = link[l:r]
    date = datetime.strptime(txt, '%Y-%m-%d')
    return date

def get_games(link : str):
    a = requests.get(link)
    tables = BeautifulSoup(a.text).find_all('div', class_ = 'blk crd lineup')
    return tables

def get_csv_tuples_from_game(game : Tag, dt : datetime) -> List[List[Any]]:
    time : str = game.find_all('time')
    teams : Tag = game.find_all('div', class_ = 'teams')[0]
    
    # Get team names
    short_names : ResultSet = teams.find_all('span', class_ = 'shrt')
    away_team, home_team = (short_name.text for short_name in short_names)
    
    # Get game time, and final datetime
    game_time_tag : Tag = game.find_all('div', class_ = 'weather-status')[0]
    game_time : str = game_time_tag.text.strip()
    
    clean_time_str : str = process_time(game_time[:-4].replace(' ',''))
    final_dt = datetime.combine(dt.date(), datetime.strptime(clean_time_str, '%H:%M:%S').time())
    
    # Away tm, home tm, date, time, player_tm, ....
    away_players = get_players(game, 'away', away_team)
    home_players = get_players(game, 'home', home_team)
    all_players = away_players + home_players

    for num, player in enumerate(all_players):
        all_players[num] = [away_team, home_team, final_dt, dt, game_time] + player
    
    return all_players

def get_players(game : Tag, whichTeam : str, tm_name : str):
    tm = game.find_all('div', class_ = f'blk {whichTeam}-team')[0]
    
    player_section = tm.find_all('ul', class_ = 'players nba')[0]
    player_tags = player_section.find_all('li', class_ = 'player')
    
    tuple_ends : List[List[str]] = []
    for player_tag in player_tags:
        txt : str = player_tag.text
        txt = re.sub(r'[\n ]+', ',', txt.strip())
        txt_list = txt.split(',')
        end = [tm_name, txt_list[0] + ' ' + txt_list[1], txt_list[2], txt_list[4]]
        tuple_ends.append(end)
    return tuple_ends

def process_time(tim : str) -> str:
    tim_list = tim.split(':')
    
    tim_list[1] = tim_list[1][:-1]
    if 'p' in tim.lower() and tim_list[0] != '12':
        tim_list[0] = int(tim_list[0]) + 12
    if 'a' in tim.lower() and tim_list[0] == '12':
        tim_list[0] = '00'
    tim_list.append('00')
    tim_str = ':'.join([str(i) for i in tim_list])
    return tim_str

def process_games_at_link(link : str):
    games : ResultSet = get_games(link)
    dt : datetime = process_date(link)
    
    for game in games[:1]:
        tuples : List[List[Any ]]= get_csv_tuples_from_game(game, dt)
        write_tuple(tuples)
        
process_date(link)

datetime.datetime(2023, 1, 22, 0, 0)

In [181]:
games = get_games(link)
dt = process_date(link)


In [193]:
game = games[0]
tups = get_csv_tuples_from_game(game, dt)

2:30P


In [ ]:
def write_tuples(tuples : List[List[Any]]):
    for tup in tuples:
        print(tup)

In [194]:
tups

[['LAC',
  'DAL',
  datetime.datetime(2023, 1, 22, 14, 30),
  datetime.datetime(2023, 1, 22, 0, 0),
  '2:30 PM ET',
  'LAC',
  'Terance Mann',
  'SG/SF',
  '$5.7K'],
 ['LAC',
  'DAL',
  datetime.datetime(2023, 1, 22, 14, 30),
  datetime.datetime(2023, 1, 22, 0, 0),
  '2:30 PM ET',
  'LAC',
  'Paul George',
  'SG/SF',
  '$8.5K'],
 ['LAC',
  'DAL',
  datetime.datetime(2023, 1, 22, 14, 30),
  datetime.datetime(2023, 1, 22, 0, 0),
  '2:30 PM ET',
  'LAC',
  'Kawhi Leonard',
  'SF/PF',
  '$9.4K'],
 ['LAC',
  'DAL',
  datetime.datetime(2023, 1, 22, 14, 30),
  datetime.datetime(2023, 1, 22, 0, 0),
  '2:30 PM ET',
  'LAC',
  'Marcus Morris',
  'PF/SF',
  '$5K'],
 ['LAC',
  'DAL',
  datetime.datetime(2023, 1, 22, 14, 30),
  datetime.datetime(2023, 1, 22, 0, 0),
  '2:30 PM ET',
  'LAC',
  'Ivica Zubac',
  'C',
  '$6.4K'],
 ['LAC',
  'DAL',
  datetime.datetime(2023, 1, 22, 14, 30),
  datetime.datetime(2023, 1, 22, 0, 0),
  '2:30 PM ET',
  'LAC',
  'Norman Powell',
  'SF/SG',
  '$6.5K'],
 ['LAC',


<div class="blk crd lineup" data-schedule-id="87399">
        <header class="hdr">
            <a class="icn icn-arw-right" href="#87399"></a>
            <div class="weather-status">
                <span class="icn-dome"></span>
                <time>2:30 PM ET</time>
            </div>
            <div class="teams">
                <span class="team-name nba lac">
                    <span class="lng">Los Angeles</span> 
                    <span class="shrt">LAC</span>
                    <span class="mascot">Clippers</span>
                </span>
                <span class="team-name nba dal">
                    <span class="lng">Dallas</span> 
                    <span class="shrt">DAL</span>
                    <span class="mascot">Mavericks</span>
                </span>
            </div>
        </header>
        <div class="blk lineup-content drw" data-anchor="87399" data-open="open" data-role="openable" data-transition="slide" id="">
            <div class="grad"></div>